In [1]:
import sys
import os

# Add the path to the aima-python repo to sys.path
sys.path.append(os.path.abspath('/Users/lukegosnell/Downloads/CUYear5/AI/FinalProject/aima_python'))

from aima_python.games4e import Game
from aima_python.utils4e import vector_add, MCT_Node, ucb


import random
random.seed(109)

In [2]:
class Mancala(Game):
    """Mancala game implementation."""

    def __init__(self):
        self.squares = {i for i in range(14) if i != 6 and i != 13}  # Indices of pits (excluding mancalas)
        self.initial = Board(pits=[4] * 6 + [0] + [4] * 6 + [0], to_move='P1')

    def actions(self, board):
        """Return a list of legal moves (non-empty pits on the current player's side)."""
        start, end = (0, 6) if board.to_move == 'P1' else (7, 13)
        return [i for i in range(start, end) if board.pits[i] > 0]

    def result(self, board, action):
        """Apply the move (distributing stones) and return the new board state."""
        pits = board.pits[:]
        player = board.to_move
        stones = pits[action]
        pits[action] = 0

        idx = action
        while stones > 0:
            idx = (idx + 1) % 14
            if (player == 'P1' and idx == 13) or (player == 'P2' and idx == 6):
                continue  # Skip opponent's mancala
            pits[idx] += 1
            stones -= 1

        # Capture condition
        if (player == 'P1' and 0 <= idx < 6 or player == 'P2' and 7 <= idx < 13) \
                and pits[idx] == 1 and pits[12 - idx] > 0:
            pits[6 if player == 'P1' else 13] += pits[idx] + pits[12 - idx]
            pits[idx] = pits[12 - idx] = 0

        # Determine next player
        next_to_move = player if (player == 'P1' and idx == 6 or player == 'P2' and idx == 13) else ('P2' if player == 'P1' else 'P1')

        return board.new({'pits': pits, 'to_move': next_to_move})

    def utility(self, board, player):
        """Return the game utility for the given player."""
        if self.is_terminal(board):
            score_p1 = sum(board.pits[:7])
            score_p2 = sum(board.pits[7:])
            return (1 if score_p1 > score_p2 else -1 if score_p1 < score_p2 else 0) if player == 'P1' else (-1 if score_p1 > score_p2 else 1 if score_p1 < score_p2 else 0)
        return 0

    def is_terminal(self, board):
        """Check if the game has ended."""
        return all(p == 0 for p in board.pits[:6]) or all(p == 0 for p in board.pits[7:13])

    def display(self, board):
        """Display the board state."""
        print("  " + " ".join(map(str, board.pits[12:6:-1])))
        print(f"{board.pits[13]}                  {board.pits[6]}")
        print("  " + " ".join(map(str, board.pits[:6])))

In [6]:
class Board(defaultdict):
    """A Mancala board with pits and a player to move."""

    def __init__(self, pits=None, to_move=None, **kwds):
        super().__init__(int)
        self.pits = pits or [0] * 14
        self.to_move = to_move

    def new(self, changes: dict, **kwds) -> 'Board':
        """Create a new board state with the specified changes."""
        board = Board(pits=self.pits[:], to_move=self.to_move, **kwds)
        board.__dict__.update(changes)
        return board

    def __hash__(self):
        return hash(tuple(self.pits)) + hash(self.to_move)

    def __repr__(self):
        return f"Mancala({self.pits}, {self.to_move})"

NameError: name 'defaultdict' is not defined

In [5]:
# Example game simulation
from random import choice

def random_player(game, state):
    return choice(game.actions(state))

mancala = Mancala()
state = mancala.initial
while not mancala.is_terminal(state):
    action = random_player(mancala, state)
    state = mancala.result(state, action)
    mancala.display(state)

print("Game over!")

TypeError: object.__init__() takes exactly one argument (the instance to initialize)